In [ ]:
# 📦 Step 1: Install and Import Libraries
import pandas as pd
import numpy as np
import string
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

# 📥 Step 2: Load the Dataset (from URL)
url = "https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv"
df = pd.read_table(url, header=None, names=["label", "message"])

print("Dataset loaded. Shape:", df.shape)
df.head()


Dataset loaded. Shape: (5572, 2)


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
# 🔍 Step 3: Preprocessing
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    text = text.strip()
    return text

df['cleaned'] = df['message'].apply(clean_text)
df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})  # encode labels
df[['label', 'cleaned']].head()


,label,cleaned
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in a wkly comp to win fa cup final...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...


In [ ]:
# ✂️ Step 4: Split and Vectorize
X = df['cleaned']
y = df['label_num']

# TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words='english')
X_tfidf = tfidf.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)


In [ ]:
# 🤖 Step 5: Train Naive Bayes Classifier
model = MultinomialNB()
model.fit(X_train, y_train)

# 🧪 Step 6: Evaluate
y_pred = model.predict(X_test)

print("Classification Report:\n")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))


Classification Report:

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       966
           1       1.00      0.74      0.85       149

    accuracy                           0.97      1115
   macro avg       0.98      0.87      0.91      1115
weighted avg       0.97      0.97      0.96      1115

Confusion Matrix:

[[966   0]
 [ 39 110]]


In [ ]:
# 🧠 Step 7: Predict on New Message
def predict_spam(message):
    message_clean = clean_text(message)
    vector = tfidf.transform([message_clean])
    pred = model.predict(vector)[0]
    return "SPAM 🚫" if pred == 1 else "HAM ✅"

# Test it
sample = "Congratulations! You've won a $1000 gift card. Click to claim."
predict_spam(sample)


'SPAM 🚫'

A fully working **spam classifier**

Uses TF-IDF + Naive Bayes

Trained and tested on real-world SMS spam dataset

Can predict new inputs via a function